In [1]:
from os.path import join as pathjoin
from data_processing import *
from interpretation import *
from models import *
from training import *
import tqdm

In [2]:
DATA_DIR = '/home/mlepekhin/data'
MODELS_DIR = '/home/mlepekhin/models'

In [5]:
transformer_model = 'bert-base-cased'
MAX_TOKENS = 512

BERT_MODEL_ID = 'allennlp_bert_base_cased'
CNN_MODEL_ID = 'allennlp_simple_cnn_en'

BERT_BEST_MODEL = pathjoin(MODELS_DIR, BERT_MODEL_ID, 'checkpoints', 'best.th')
CNN_BEST_MODEL = pathjoin(MODELS_DIR, CNN_MODEL_ID, 'checkpoints', 'best.th')

In [6]:
vocab = Vocabulary().from_files(pathjoin(MODELS_DIR, BERT_MODEL_ID, 'vocab'))
model = build_transformer_model(vocab, transformer_model)
model.load_state_dict(torch.load(BERT_BEST_MODEL))

bert_predictor = TextClassifierPredictor(
    model, 
    dataset_reader=build_transformer_dataset_reader(transformer_model, lower=True)
)

Building the model


In [7]:
cnn_vocab = Vocabulary().from_files(pathjoin(MODELS_DIR, CNN_MODEL_ID, 'vocab'))
cnn_model = build_simple_cnn_model(
    cnn_vocab, emb_size=256, output_dim=128, num_filters=32, ngram_filter_sizes=(2, 3, 4, 5)
)
cnn_model.load_state_dict(torch.load(CNN_BEST_MODEL))

cnn_predictor = TextClassifierPredictor(
    cnn_model, 
    dataset_reader=build_dataset_reader(None, lower=True)
)

Building the model


In [8]:
test_df = pd.read_csv(pathjoin(DATA_DIR, 'ru_test'))
test_df.head()

,Unnamed: 0,target,text
0,726,A7,Глава 1 Приступая к работе 1.1 Знакомство с те...
1,1871,A17,Kawasaki D-Tracker С недавних пор Kawasaki d-t...
2,1265,A17,"По моему , вполне достойные книги , может и не..."
3,205,A11,Тест-драйв Lada Granta : новая надежда автогра...
4,141,A8,"среда , 2 декабря 2009 года , 12.33 Бумага всё..."


In [9]:
index_to_token = vocab.get_index_to_token_vocabulary('labels')
token_to_index = vocab.get_token_to_index_vocabulary('labels')
print(index_to_token)
print(cnn_vocab.get_index_to_token_vocabulary('labels'))

{0: 'A1', 1: 'A12', 2: 'A7', 3: 'A16', 4: 'A8', 5: 'A22', 6: 'A4', 7: 'A11', 8: 'A14', 9: 'A9', 10: 'A17'}
{0: 'A1', 1: 'A12', 2: 'A7', 3: 'A16', 4: 'A8', 5: 'A22', 6: 'A4', 7: 'A11', 8: 'A14', 9: 'A9', 10: 'A17'}


In [10]:
probs_bert, probs_cnn = [], []

for text, target in tqdm.tqdm(zip(test_df.text.values, test_df.target.values)):
    probs_bert.append(bert_predictor.predict(text)['probs'])
    probs_cnn.append(cnn_predictor.predict(text)['probs'])
probs_bert = np.array(probs_bert)
probs_cnn = np.array(probs_cnn)

483it [02:10,  3.72it/s]


In [11]:
probs_mix = 0.5 * probs_bert + 0.5 * probs_cnn
predicted_bert = [index_to_token[np.argmax(vec)] for vec in probs_bert]
predicted_cnn = [index_to_token[np.argmax(vec)] for vec in probs_cnn]
predicted_mix = [index_to_token[np.argmax(vec)] for vec in probs_mix]

In [12]:
calc_classifier_metrics(np.array(predicted_bert), test_df.target.values)

label (argum) f1_score 0.0 precision 0.0 recall 0.0
label (fictive) f1_score 0.0 precision 0.0 recall 0.0
label (instruct) f1_score 0.0 precision 0.0 recall 0.0
label (reporting) f1_score 0.0 precision 0.0 recall 0.0
label (legal) f1_score 0.0 precision 0.0 recall 0.0
label (personal) f1_score 0.0 precision 0.0 recall 0.0
label (commercial) f1_score 0.0 precision 0.0 recall 0.0
label (research) f1_score 0.0 precision 0.0 recall 0.0
label (info) f1_score 0.07142857142857144 precision 0.06060606060606061 recall 0.08695652173913043
label (eval) f1_score 0.0 precision 0.0 recall 0.0
accuracy 0.004140786749482402


/home/mlepekhin/anaconda3/envs/mlepekhin_research/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [13]:
calc_classifier_metrics(np.array(predicted_cnn), test_df.target.values)

label (argum) f1_score 0.07407407407407407 precision 0.03896103896103896 recall 0.75
label (fictive) f1_score 0.0 precision 0.0 recall 0.0
label (instruct) f1_score 0.0 precision 0.0 recall 0.0
label (reporting) f1_score 0.3956043956043956 precision 0.5242718446601942 recall 0.3176470588235294
label (legal) f1_score 0.0 precision 0.0 recall 0.0
label (personal) f1_score 0.0 precision 0.0 recall 0.0
label (commercial) f1_score 0.22058823529411767 precision 0.17647058823529413 recall 0.29411764705882354
label (research) f1_score 0.09836065573770492 precision 0.061224489795918366 recall 0.25
label (info) f1_score 0.16727272727272727 precision 0.696969696969697 recall 0.09504132231404959
label (eval) f1_score 0.0 precision 0.0 recall 0.0
accuracy 0.2028985507246377


In [14]:
calc_classifier_metrics(np.array(predicted_mix), test_df.target.values)

label (argum) f1_score 0.0 precision 0.0 recall 0.0
label (fictive) f1_score 0.0 precision 0.0 recall 0.0
label (instruct) f1_score 0.0 precision 0.0 recall 0.0
label (reporting) f1_score 0.0 precision 0.0 recall 0.0
label (legal) f1_score 0.0 precision 0.0 recall 0.0
label (personal) f1_score 0.0 precision 0.0 recall 0.0
label (commercial) f1_score 0.0 precision 0.0 recall 0.0
label (research) f1_score 0.0 precision 0.0 recall 0.0
label (info) f1_score 0.0967741935483871 precision 0.09090909090909091 recall 0.10344827586206896
label (eval) f1_score 0.0 precision 0.0 recall 0.0
accuracy 0.006211180124223602


In [101]:
import pickle
from os.path import join as pathjoin

def save_model(predictor, vectorizer, model_dir):
    !mkdir {model_dir}
    with open(pathjoin(model_dir, 'predictor'), 'wb') as fout:
        fout.write(pickle.dumps(predictor))
    with open(pathjoin(model_dir, 'vectorizer'), 'wb') as fout:
        fout.write(pickle.dumps(vectorizer))
        
def load_model(model_dir):
    return pickle.loads(open(pathjoin(model_dir, 'predictor'), 'rb').read()),\
           pickle.loads(open(pathjoin(model_dir, 'vectorizer'), 'rb').read())

In [106]:
from scipy import sparse

class BigVectorizer:
    def __init__(self, max_word_features=5000, max_char_features=10000):
        self.vect_word = TfidfVectorizer(
            max_features=max_word_features, lowercase=True, analyzer='word',
            stop_words=stopwords.words('russian'), ngram_range=(1,3),dtype=np.float32
        )
        self.vect_char = TfidfVectorizer(
            max_features=max_char_features, lowercase=True, analyzer='char',
            stop_words=stopwords.words('russian'), ngram_range=(3,6),dtype=np.float32
        )

    def fit_transform(self, X):
        vect_word = self.vect_word.fit_transform(X)
        vect_char = self.vect_char.fit_transform(X)
        return sparse.hstack([vect_word, vect_char])
       
    def transform(self, X):
        vect_word = self.vect_word.transform(X)
        vect_char = self.vect_char.transform(X)
        return sparse.hstack([vect_word, vect_char])

In [103]:
lr_predictor, vectorizer = load_model('simple_lr')

In [109]:
lr_proba = lr_predictor.predict_proba(vectorizer.transform(test_df.text))

In [110]:
predicted_lr = lr_predictor.predict(vectorizer.transform(test_df.text))

In [111]:
calc_classifier_metrics(np.array(predicted_lr), test_df.target.values)

label (argum) f1_score 0.7100591715976331 precision 0.7792207792207793 recall 0.6521739130434783
label (fictive) f1_score 0.7567567567567568 precision 0.6086956521739131 recall 1.0
label (instruct) f1_score 0.6923076923076924 precision 0.5294117647058824 recall 1.0
label (reporting) f1_score 0.9124423963133641 precision 0.9611650485436893 recall 0.868421052631579
label (legal) f1_score 0.9230769230769231 precision 0.9230769230769231 recall 0.9230769230769231
label (personal) f1_score 0.6732673267326732 precision 0.6938775510204082 recall 0.6538461538461539
label (commercial) f1_score 0.9239766081871345 precision 0.9294117647058824 recall 0.9186046511627907
label (research) f1_score 0.8200000000000001 precision 0.8367346938775511 recall 0.803921568627451
label (info) f1_score 0.39999999999999997 precision 0.2727272727272727 recall 0.75
label (eval) f1_score 0.7567567567567567 precision 0.8235294117647058 recall 0.7
accuracy 0.7971014492753623


In [116]:
index_to_index = {}
for class_id, cl in enumerate(lr_predictor.classes_):
    index_to_index[class_id] = token_to_index[cl]

lr_proba_transformed = []
for vec in lr_proba:
    new_vec = np.zeros(len(vec))
    for i in range(len(vec)):
        new_vec[index_to_index[i]] = vec[i]
    lr_proba_transformed.append(new_vec[:])
lr_proba_transformed = np.array(lr_proba_transformed)

In [131]:
probs_mix_lr = 0.3 * probs_bert + 0.4 * lr_proba_transformed + 0.3 * probs_cnn
predicted_mix_lr = [index_to_token[np.argmax(vec)] for vec in probs_mix_lr]
calc_classifier_metrics(np.array(predicted_mix_lr), test_df.target.values)

label (argum) f1_score 0.7272727272727272 precision 0.7792207792207793 recall 0.6818181818181818
label (fictive) f1_score 0.8292682926829269 precision 0.7391304347826086 recall 0.9444444444444444
label (instruct) f1_score 0.8387096774193549 precision 0.7647058823529411 recall 0.9285714285714286
label (reporting) f1_score 0.9573459715639812 precision 0.9805825242718447 recall 0.9351851851851852
label (legal) f1_score 0.8148148148148148 precision 0.8461538461538461 recall 0.7857142857142857
label (personal) f1_score 0.7368421052631581 precision 0.7142857142857143 recall 0.7608695652173914
label (commercial) f1_score 0.9404761904761904 precision 0.9294117647058824 recall 0.9518072289156626
label (research) f1_score 0.8627450980392157 precision 0.8979591836734694 recall 0.8301886792452831
label (info) f1_score 0.5306122448979591 precision 0.3939393939393939 recall 0.8125
label (eval) f1_score 0.7532467532467532 precision 0.8529411764705882 recall 0.6744186046511628
accuracy 0.8322981366459